In [71]:
import numpy as np
import pandas as pd

In [73]:
# Different temperature sets - data segregration

index = [];
start = [];
end   = [];
TSet  = [];

for i in range(len(DataT1)-1):
    if (DataT1[i] != DataT1[i+1]):
        start.append(i+1);
        end.append(i);
        TSet.append(DataT1[i+1]);
        
start.insert(0,0);
end.insert(len(end),len(DataT1));
TSet.insert(0,(DataT1[0]));

[0, 1904, 3814, 5611]
[1903, 3813, 5610, 7400]
[25, 50, 70, 90]


In [74]:
# load the dataset
DataF = DataF1[start[m]:end[m]+1];
DataB = DataB1[start[m]:end[m]+1];
DataT = DataT1[start[m]:end[m]+1];
DataIndex = DataIndex1[start[m]:end[m]+1];

In [75]:
## slope, normalized slope; absolute slope estimation for estimating absolute maximum for 
## estimating critical points

slope      = [[]];
norm_slope = [[]];
max_slope  = [];
abs_slope  = [[]];

for i in range(len(DataB[:][:])):
    slope.append([]);
    abs_slope.append([]);
    norm_slope.append([]);

for i in range(len(DataB[:][:])):
    for j in range(len(DataB[1])-1):
        slope[i].append(DataB[i][j+1]-DataB[i][j]);
        abs_slope[i].append(abs(slope[i][j]));
    max_slope.append(max(abs_slope[i][:]));

for i in range(len(DataB[:][:])):
    for j in range(len(DataB[1])-1):
        norm_slope[i].append(slope[i][j]/max_slope[i]);

In [76]:
db_slope = [[]];
db_slope_norm = [[]];
max_db_slope  = [];
abs_db_slope = [[]]; abs_db_slope_norm = [[]];

for i in range(len(DataB[:][:])):
    db_slope_norm.append([]);
    db_slope.append([]);
    abs_db_slope.append([]);
    abs_db_slope_norm.append([]);

for i in range(len(DataB[:][:])):
    for j in range(len(DataB[1])-2):
        db_slope[i].append(norm_slope[i][j+1] - norm_slope[i][j]);
        abs_db_slope[i].append(abs(db_slope[i][j]));
    max_db_slope.append(max(abs_db_slope[i][:]));

for i in range(len(DataB[:][:])):
    for j in range(len(DataB[1])-2):
        db_slope_norm[i].append((db_slope[i][j]/max_db_slope[i]));
        abs_db_slope_norm[i].append(abs(db_slope_norm[i][j]));

In [77]:
final = [[]];    output = [[]];
critical = [[]]; critical_1 = [[]];

for i in range(len(DataB[:])):
    final.append([]);
    output.append([]);
    critical.append([]);
    critical_1.append([]);

## Selection of slope range greater some threshold values
for i in range(len(DataB[:])):
    for j in range(len(db_slope_norm[1])):
        x = abs(db_slope_norm[i][j]);
        y = abs(db_slope[i][j]);
        # print(x, avg)
        if ((x > 0.2) & (y > 0.02)):
            final[i].append(j)

for i in range(len(DataB[:])):
    try:
        prev = final[i][0];
        j = 1;
        while (j < len(final[i])):
            if (final[i][j-1] == (final[i][j] - 1)):
                j +=1;
            else :
                output[i].append((prev, final[i][j-1]))
                prev = final[i][j];
                j+=1;
        if (prev != final[i][-1]):
            output[i].append((prev, final[i][-1]))
        if (prev == final[i][-1]):
            output[i].append((prev, final[i][-1]))
    except :
        output[i].append(0);

## Estimating the peaks in the range

length1=[]; length=[];
for i in range(0,len(DataB[:])):
    for peak in output[i]:
        if (len(output[i]) == 1):
            critical[i].append(0);    
        else:
            maxi = max(abs_db_slope_norm[i][peak[0]:peak[1]+1]);
            critical[i].append(abs_db_slope_norm[i][peak[0]:peak[1]+1].index(maxi) + peak[0]);

for i in range(len(DataB[:])):
    for j in range(len(critical[i])):
        critical_1[i].append(0);

for i in range(len(DataB[:])):
    for j in range(len(critical[i])):
        if ((len(critical[i]) == 1)):
            critical_1[i][j] = -1;
            break;
            
        if (abs(critical[i][j] - critical[i][j-1]) > 50):
            critical_1[i][j] = critical[i][j];
        elif (abs(critical[i][j] - critical[i][j-1]) < 50):
            critical_1[i][j-1] = (critical[i][j]);
            if (abs(critical[i][j-1] - critical[i][j-2]) < 50):
                critical_1[i][j-2] = (critical[i][j]);
                critical_1[i][j-1] = 0;
                # if (abs(critical[i][j-2] - critical[i][j-3]) < 50):
                #     critical_1[i][j-3] = (critical[i][j]);
                #     critical_1[i][j-2] = 0;
                #     critical_1[i][j-1] = 0;
            
    critical_1[i] = [x for x in critical_1[i] if x != 0];

for i in range(len(critical[i])):
    if(len(critical_1[i])>1):
        p = critical_1[i][-2];
        q = critical_1[i][-1];
        if(p>q):
            critical_1[i] = np.delete(critical_1[i],-1);
         
for i in range(len(DataB[:])):
    length1.append(len(critical_1[i]));

In [79]:
lenB = len(DataB[:]);
sineB = [[]]; trigB = [[]]; trapB = [[]]; trigsymB = [[]];
sinef = [0]*lenB; trigf = [0]*lenB; trapf = [0]*lenB; trigsymf = [0]*lenB;
sineT = [0]*lenB; trigT = [0]*lenB; trapT = [0]*lenB; trigsymT = [0]*lenB;
crittrap = [[]]; crittrig = [[]]; crittrigsym = [[]];
duty = []; duty1 = [];
D1 = []; D2 = []; D3 = []; D4 = []; 
sineInd=[0]*lenB; trigInd=[0]*lenB; trapInd=[0]*lenB;
 
for i in range(len(DataB[:])):
    sineB.append([]);
    trigB.append([]);
    trigsymB.append([]);
    trapB.append([]);
    crittrig.append([]);
    crittrap.append([]);
    crittrigsym.append([]);

## Data segregration into sinusoidal, triangular and trapezoidal waveforms
for i in range(len(DataB[:][:])):
    if(len(critical_1[i]) == 2): 
        duty1.append((critical_1[i][0]+critical_1[i][1])/1024 -1);
        duty.append(round((critical_1[i][0]+critical_1[i][1])/1024 -1,1));
        crittrig[i].append(critical_1[i]);
    if(len(critical_1[i]) == 4):
        D1.append(round((critical_1[i][0])/1024,1));
        D2.append(round((critical_1[i][1] - critical_1[i][0])/1024,1));
        D3.append(round((critical_1[i][2] - critical_1[i][1])/1024,1));
        D4.append(round((critical_1[i][3] - critical_1[i][2])/1024,1));
        crittrap[i].append(critical_1[i]);
    for j in range(len(DataB[i])):
        if(len(critical_1[i]) == 1): 
            sineB[i].append(DataB[i][j]);
            sinef[i]= (DataF[i]);
            sineT[i]= DataT[i];
            sineInd[i] = DataIndex[i];
        if(len(critical_1[i]) == 2): 
            trigB[i].append(DataB[i][j]);
            trigf[i]= (DataF[i]);
            trigT[i]= DataT[i];
            trigInd[i] = DataIndex[i];
        if(len(critical_1[i]) == 4): 
            trapB[i].append(DataB[i][j]);
            trapf[i]= (DataF[i]);
            trapT[i]= DataT[i];
            trapInd[i] = DataIndex[i];


## Removing empty and zero entries in the array.
sineB = [x for x in sineB if x != []]
sinef = [x for x in sinef if x != 0]
sineT = [x for x in sineT if x != 0]
sineInd = [x for x in sineInd if x != 0]

trigB = [x for x in trigB if x != []]
trigf = [x for x in trigf if x != 0]
trigT = [x for x in trigT if x != 0]
trigInd = [x for x in trigInd if x != 0]

trapB = [x for x in trapB if x != []]
trapf = [x for x in trapf if x != 0]
trapT = [x for x in trapT if x != 0]
trapInd = [x for x in trapInd if x != 0]

crittrig = [x for x in crittrig if x != []]
crittrap = [x for x in crittrap if x != []]

## Symmetrical triangular waveform segregation
for i in range(len(trigB[:])):
    if(duty[i] == 0.5): 
        crittrigsym[i].append(critical_1[i]);        
    for j in range(len(trigB[i])):
        if(duty[i] == 0.5): 
            trigsymB[i].append(trigB[i][j]);
            trigsymf[i]= trigf[i];
            trigsymT[i]= trigT[i];

trigsymB = [x for x in trigsymB if x != []]
trigsymf = [x for x in trigsymf if x != 0]
trigsymT = [x for x in trigsymT if x != 0]
crittrigsym = [x for x in crittrigsym if x != []]

In [80]:
sineBpk = []; trigBpk = []; trapBpk = [];
sineMinB = []; sineMaxB=[]; sineRMS = []; sinePeriod = [];

for i in range(0,len(sineB)):
    sineMaxB.append(max(sineB[i]));
    sineMinB.append(abs(min(sineB[i])));
    sineBpk.append((max(sineB[i])-min(sineB[i])));
    sinePeriod.append(1/sinef[i])
    time = (np.arange(0,len(sineB[i]),1))*(1/sinef[i])
    rms = np.sqrt(np.trapz(np.multiply(sineB[i],sineB[i]),time)/(time[-1]-time[0]))
    sineRMS.append(rms)

sineBpk = (np.array(sineBpk)).flatten(); 

for i in range(0,len(trigB)):
    maxB = max(trigB[i]);
    minB = min(trigB[i]);
    trigBpk.append(0.5*(maxB-minB));

trigBpk = (np.array(trigBpk)).flatten(); 

for i in range(0,len(trapB)):
    maxB = max(trapB[i]);
    minB = min(trapB[i]);
    trapBpk.append(0.5*(maxB-minB));

trapBpk = (np.array(trapBpk)).flatten(); 